In [ ]:
# change name of columns to lower case and replace spaces with underscores
df.columns = df.columns.str.lower().str.replace(' ', '_')

# change text of strig columns to lower case and replace spaces with underscores
# TO IMPLEMENT
strings = list(df.dtypes[df.dtypes == 'object'].index)
for col in strings:
    df[col] = df[col].str.lower().str.replace(' ', '_')

# check the unique values in columns
for col in df.columns:
    print(col)
    print(df[col].unique()[:5]) # print first 5 unique values
    print(df[col].nunique()) # print number of unique values
    print()


# check missing values

# check target variable distribution
# apply log if the target variable is skewed
# do not forget to treat log 0 as adding 1, since log 0  is undefined
price_logs = np.log1p(df.msrp) # you can apply log1p to treat log 0

## Check relationship between variables

- **FOR CATEGORICAL** Use mutual_info_score
- **FOR NUMERICAL** Use pearson (note: only linear relationships) / Strong if r>0.6, moderate between 0.2 & 0.5

| Feature               | Mutual Information (MI)        | Correlation (e.g., Pearson)   |
|-----------------------|---------------------------------|--------------------------------|
| **Type of Relationship** | Measures **any** kind of dependency (linear & nonlinear) | Measures **only linear** relationships |
| **Range of Values**   | Always **≥ 0** (0 = no relation, higher = stronger dependency) | Between **-1 and 1** (-1 = strong negative, 1 = strong positive, 0 = no correlation) |
| **Interpretation**    | Measures **how much knowing one variable reduces uncertainty** about the other | Measures **how much one variable changes proportionally** with another |
| **Handles Categorical Data?** | ✅ Yes (works for both categorical & numerical data) | ❌ No (only works well for numerical data) |
| **Sensitivity to Scale** | Not affected by scaling | Strongly affected by scaling (needs normalization) |
